In [11]:

from typing import Iterator, List
from pdfminer.high_level import extract_pages
from pdfminer.layout import LTTextContainer, LTPage
from collections import Counter

def read_file(file_name):
    file = extract_pages(file_name)
    return file
    
    
def get_paragraphs(file: Iterator[LTPage]) -> List[dict]:
    paragraphs = []
    page_num = 0
    paragraph_id = 0
    for page_layout in file:
        page_num += 1
        
        for element in page_layout:
            if isinstance(element, LTTextContainer):
                paragraphs.append({
                    'id': paragraph_id,
                    'page': page_num,
                    'paragraph': element.get_text()
                })
                paragraph_id += 1
                
    return paragraphs

def get_terms(file: Iterator[LTPage]) -> List[dict]:
    pass


1: CHAPTER



2: The Rise of Platform Ecosystems 1



3: May you live in interesting times.
Ancient Chinese curse



4: IN THIS CHAPTER



5: (cid:129) What platforms are and what they are not



6: (cid:129) Core components of platform ecosystems



7: (cid:129) Drivers of the migration toward platforms in diverse industries



8: 1.1 THE WAR OF ECOSYSTEMS
Blackberry had everything going right. It had fanatically loyal customers and its products were inno-
vative, well engineered, durable, and got raving reviews from critics. After years of commanding a
lion’s share (about 50%) of the smartphone market that it largely created, it had trouble breaking past
a 1% market share with its newest products by 2012, leading to its subsequent downfall Blackberry
assumed that the problem was Apple and then Google—both industry outsiders—who had since
entered the fray. So, it did what made sense: Price more competitively, invest more in developing
new products, upgrade its operating system, and st

In [ ]:
in_file = read_file('../data/raw/Chapter-1---The-Rise-of-Platform-Ecosystems_2014_Platform-Ecosystems.pdf')
paragraphs = get_paragraphs(in_file)

for i in range(10):
    print(f"{i + 1}: {paragraphs[i]['paragraph']}\n\n")